# Preprocesamiento

## Lectura de datos

In [29]:
import pandas as pd
import nltk
import gensim

df_true = pd.read_csv("./dataset/True.csv")
df_fake = pd.read_csv("./dataset/Fake.csv")

#### Agregar columna de identificación de Fake News

In [30]:
df_true['isfake'] = 1

df_true.head()

,title,text,subject,date,isfake
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [31]:
df_fake['isfake'] = 0

df_fake.head()

,title,text,subject,date,isfake
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [32]:
df = pd.concat([df_true, df_fake]).reset_index(drop = True)

### Combinación de título y texto

In [33]:
df['original'] = df['title'] + '. ' + df['text']
df['original'].replace("..", ".")
df.head()

,title,text,subject,date,isfake,original
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1,"As U.S. budget fight looms, Republicans flip t..."
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1,U.S. military to accept transgender recruits o...
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1,Senior U.S. Republican senator: 'Let Mr. Muell...
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1,FBI Russia probe helped by Australian diplomat...
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1,Trump wants Postal Service to charge 'much mor...


#### Muestra de la combinación anterior

In [34]:
df['title'][0]

'As U.S. budget fight looms, Republicans flip their fiscal script'

In [35]:
df['text'][0]

'WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS’ “Face the Nation,” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for non-defense “discretionary” spending on programs that support educat

In [36]:
df['original'][0]

'As U.S. budget fight looms, Republicans flip their fiscal script. WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS’ “Face the Nation,” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for no

In [37]:
import os

if os.path.exists(r'dataset/lemmatized.csv'):
    df = pd.read_csv(r'dataset/lemmatized.csv')
else:
    import stanza, torch, gc
    from stanza.models.common.doc import Document
    nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma')

    def token_lemma(_text):
        tokend = []
        tokens_lemmatized = []
        stanza_doc = stanza.Document([], text=_text)
        stanza_doc = nlp(stanza_doc)
        for sent in stanza_doc.sentences:
            for word in sent.words:
                tokens_lemmatized.append(word.lemma)
        return tokens_lemmatized
    
    df['lemma'] = df['original'].apply(token_lemma)
        
    df.to_csv(r'dataset/lemmatized.csv')

In [38]:
df.head()

,Unnamed: 0,title,text,subject,date,isfake,original,lemma
0,0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1,"As U.S. budget fight looms, Republicans flip t...","['as', 'U.S.', 'budget', 'fight', 'loom', ',',..."
1,1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1,U.S. military to accept transgender recruits o...,"['U.S.', 'military', 'to', 'accept', 'transgen..."
2,2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1,Senior U.S. Republican senator: 'Let Mr. Muell...,"['senior', 'U.S.', 'republican', 'senator', ':..."
3,3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1,FBI Russia probe helped by Australian diplomat...,"['FBI', 'Russia', 'probe', 'help', 'by', 'Aust..."
4,4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1,Trump wants Postal Service to charge 'much mor...,"['Trump', 'want', 'postal', 'Service', 'to', '..."


----

### Descargar Stop-Words

In [39]:
# nltk.download("stopwords")

In [40]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
stop_words.extend(list(gensim.parsing.preprocessing.STOPWORDS))
stop_words = list(set(stop_words))


In [41]:
for word in stop_words:
    print(word, end=', ')

he, hereupon, mostly, since, too, amount, doing, using, everywhere, we, may, latterly, those, afterwards, very, empty, something, around, they, becomes, once, thin, found, being, with, make, hundred, were, has, five, below, where, well, hasn, cannot, besides, detail, whether, through, yourselves, move, that'll, mustn, it, bill, others, noone, wherein, keep, i, eight, t, as, edu, could, itself, at, anyone, one, beside, go, find, already, into, these, to, mightn, hadn't, nothing, you, wasn, own, front, should've, needn't, alone, anything, hers, an, their, o, enough, this, among, during, serious, yet, mustn't, thereafter, down, almost, did, above, whatever, here, are, shouldn't, quite, nine, of, again, shouldn, didn't, across, became, anyhow, least, shan't, himself, but, you've, weren, thereupon, meanwhile, former, it's, will, in, regarding, therein, same, used, my, mill, wouldn't, whose, mightn't, that, was, see, put, sometimes, couldn't, because, isn, co, more, back, although, aren, two

### Eliminar Stop-Words del dataset

In [42]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if (len(token) > 3 and token not in stop_words) or '.' in token or ',' in token:
            result.append(token)
            
    return result

In [43]:
df['clean'] = df['lemma'].apply(preprocess)
df.head()

,Unnamed: 0,title,text,subject,date,isfake,original,lemma,clean
0,0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1,"As U.S. budget fight looms, Republicans flip t...","['as', 'U.S.', 'budget', 'fight', 'loom', ',',...","[budget, fight, loom, republicans, flip, fisca..."
1,1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1,U.S. military to accept transgender recruits o...,"['U.S.', 'military', 'to', 'accept', 'transgen...","[military, accept, transgender, recruit, monda..."
2,2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1,Senior U.S. Republican senator: 'Let Mr. Muell...,"['senior', 'U.S.', 'republican', 'senator', ':...","[senior, republican, senator, mueller, washing..."
3,3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1,FBI Russia probe helped by Australian diplomat...,"['FBI', 'Russia', 'probe', 'help', 'by', 'Aust...","[russia, probe, help, australian, diplomat, wa..."
4,4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1,Trump wants Postal Service to charge 'much mor...,"['Trump', 'want', 'postal', 'Service', 'to', '...","[trump, want, postal, service, charge, amazon,..."


### Ejemplo de registro después de remover Stop-Words

In [44]:
print(df['clean'][0])

['budget', 'fight', 'loom', 'republicans', 'flip', 'fiscal', 'script', 'washington', 'reuters', 'head', 'conservative', 'republican', 'faction', 'congress', 'vote', 'month', 'huge', 'expansion', 'national', 'debt', 'fiscal', 'conservative', 'sunday', 'urge', 'budget', 'restraint', 'sharp', 'pivot', 'republicans', 'representative', 'mark', 'meadows', 'speak', 'face', 'nation', 'draw', 'hard', 'line', 'federal', 'spending', 'lawmaker', 'brace', 'battle', 'january', 'return', 'holiday', 'wednesday', 'lawmaker', 'begin', 'pass', 'federal', 'budget', 'fight', 'likely', 'link', 'issue', 'immigration', 'policy', 'november', 'congressional', 'election', 'campaign', 'approach', 'republicans', 'seek', 'control', 'congress', 'president', 'donald', 'trump', 'republicans', 'want', 'budget', 'increase', 'military', 'spending', 'democrats', 'want', 'proportional', 'increase', 'defense', 'discretionary', 'spend', 'program', 'support', 'education', 'scientific', 'research', 'infrastructure', 'public', 

### Total de palabras en el Dataset

In [54]:
list_of_words = []
unique_words = set()
for document in df.clean:
    for word in document:
        list_of_words.append(word)
        unique_words.add(word)
        
total_words = len(list_of_words)  # total words
unique_words = len(unique_words)   # total unique words
print("Total words:" + str(total_words) + " unique_words:" + str(unique_words))

Total words:8844981 unique_words:96148


Unir palabras nuevamente 

In [46]:
df['clean_joined'] = df['clean'].apply(lambda x: " ".join(x))

df.head()

,Unnamed: 0,title,text,subject,date,isfake,original,lemma,clean,clean_joined
0,0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1,"As U.S. budget fight looms, Republicans flip t...","['as', 'U.S.', 'budget', 'fight', 'loom', ',',...","[budget, fight, loom, republicans, flip, fisca...",budget fight loom republicans flip fiscal scri...
1,1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1,U.S. military to accept transgender recruits o...,"['U.S.', 'military', 'to', 'accept', 'transgen...","[military, accept, transgender, recruit, monda...",military accept transgender recruit monday pen...
2,2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1,Senior U.S. Republican senator: 'Let Mr. Muell...,"['senior', 'U.S.', 'republican', 'senator', ':...","[senior, republican, senator, mueller, washing...",senior republican senator mueller washington r...
3,3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1,FBI Russia probe helped by Australian diplomat...,"['FBI', 'Russia', 'probe', 'help', 'by', 'Aust...","[russia, probe, help, australian, diplomat, wa...",russia probe help australian diplomat washingt...
4,4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1,Trump wants Postal Service to charge 'much mor...,"['Trump', 'want', 'postal', 'Service', 'to', '...","[trump, want, postal, service, charge, amazon,...",trump want postal service charge amazon shipme...


### Ejemplo de texto limpio

In [49]:
df['clean_joined'][0]

'budget fight loom republicans flip fiscal script washington reuters head conservative republican faction congress vote month huge expansion national debt fiscal conservative sunday urge budget restraint sharp pivot republicans representative mark meadows speak face nation draw hard line federal spending lawmaker brace battle january return holiday wednesday lawmaker begin pass federal budget fight likely link issue immigration policy november congressional election campaign approach republicans seek control congress president donald trump republicans want budget increase military spending democrats want proportional increase defense discretionary spend program support education scientific research infrastructure public health environmental protection trump administration willing increase defense discretionary spending percent meadows chairman small influential house freedom caucus program democrats need government raise percent fiscal conservative rationale eventually people money mea

## Guardar csv del dataset con los cambios 

In [56]:
df.to_pickle(r'dataset/final.pkl')